In [ ]:
# default_exp data.finance

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from libs import is_lib_exists

In [5]:
required_libs  = ["numpy", "pandas", "seaborn", "matplotlib", "yfinance"]

In [7]:
for lib in required_libs:
    if not is_lib_exists(lib):
        print(f"Installing {lib}")
    else:
        print(f"{lib} exists")

numpy exists
pandas exists
Installing seaborn
Installing matplotlib
yfinance exists


In [8]:
#export
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'

# Finance Data

> Functionality to gather finance data.

The classes here provide bases classes used to gather finance data for model training.

In [9]:
#hide
from nbdev.showdoc import *